In [1]:
import pandas as pd
import numpy as np
import csv
import requests
import urllib.request, json 
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Convenience function for transfermarkt data
def strToDataframe(x):
    x = x.strip()
    firstWord = x[0:x.find(' ')]
    b = x[int(x.find('€'))+1:]
    b = b[int(b.find('€'))+1:]
    b = b[:b.find('mil')]
    c = x[x.find(',') - 2: x.find(',')+2]
    b = b.replace(",", ".")
    c = c.replace(",", ".")
    d = {'Team': [firstWord.upper()], 'TeamWorth': [b], 'TeamAge': [c]}
    stored = pd.DataFrame(data=d)
    return stored

In [11]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.com.tr/super-lig/startseite/wettbewerb/TR1/plus/1?saison_id=2015"
pageTree = requests.get(page, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'lxml')
odd = pageSoup.find_all("tr", {"class": "odd"})
even = pageSoup.find_all("tr", {"class": "even"})
#Let's look at the first name in the Players list.
EvenDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
EvenData = pd.DataFrame(data=EvenDat)
OddDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
OddData = pd.DataFrame(data=OddDat)
FullDat = {'Team': [], 'TeamWorth': [], 'TeamAge': []}
FullData = pd.DataFrame(data=FullDat)
for Even in even:
    EvenData = pd.concat([EvenData, strToDataframe(Even.text)])
for Odd in odd:
    OddData = pd.concat([OddData, strToDataframe(Odd.text)])
FullData = pd.concat([OddData, EvenData])
FullData.TeamAge = pd.to_numeric(FullData.TeamAge, errors='coerce')
FullData.TeamWorth = pd.to_numeric(FullData.TeamWorth, errors='coerce')
FullData = FullData.dropna()
FullData
FullData.index = FullData['Team']
FullData['Support'] = [28589, 18668, 11245, 1950, 2716, 2565, 3065, 4251, 3314, 18996, 7696, 9362, 6815, 4186, 4881, 2394, 15871, 5116]
FullData

,Team,TeamAge,TeamWorth,Support
Team,,,,
FENERBAHÇE,FENERBAHÇE,24.5,195.38,28589
BEŞIKTAŞ,BEŞIKTAŞ,24.5,135.30,18668
BURSASPOR,BURSASPOR,23.9,89.38,11245
KASIMPAŞA,KASIMPAŞA,25.1,48.70,1950
BAŞAKŞEHIR,BAŞAKŞEHIR,26.3,48.00,2716
GENÇLERBIRLIĞI,GENÇLERBIRLIĞI,25.4,42.85,2565
GAZIANTEPSPOR,GAZIANTEPSPOR,23.5,36.68,3065
KAYSERISPOR,KAYSERISPOR,24.4,35.55,4251
ÇAYKUR,ÇAYKUR,24.3,32.63,3314


In [12]:
data = []
with urllib.request.urlopen('https://raw.githubusercontent.com/jokecamp/FootballData/master/Turkey/SuperLig/2015-2016/superlig.json') as url:
    data = json.loads(url.read().decode())
#d = {'week': [data[0]['week']], 'home': [data[0]['matches'][0]['homeTeam']['name']], 'away': [data[0]['matches'][0]['awayTeam']['name']], 'score': [data[0]['matches'][0]['match']['score']] }
d = {'week': [], 'home': [], 'away': [], 'score': [], 'winner': [], 'Difference': [], 'Season': []}
df = pd.DataFrame(data=d)
print(df)
for x in data:
    for y in x['matches']:
        result = 'draw'
        
        if (int([y['match']['score']][0][0]) > int([y['match']['score']][0][-1])):
            result = 'home'
        elif (int([y['match']['score']][0][0]) < int([y['match']['score']][0][-1])):
            result = 'away'
        tem = {'week': [int(x['week'][0:x['week'].index('.')])], 'home': [y['homeTeam']['name']], 'away': [y['awayTeam']['name']], 'score': [y['match']['score']], 'winner': [result], 'Difference': [(int([y['match']['score']][0][0]) - int([y['match']['score']][0][-1]))], 'Season': [2015]}
        temp = pd.DataFrame(data=tem)
        df = pd.concat([df, temp])      
ChgOfI = lambda x: x.replace('İ', 'I')
df['home'] = df['home'].apply(ChgOfI)
df['away'] = df['away'].apply(ChgOfI)

Empty DataFrame
Columns: [Difference, Season, away, home, score, week, winner]
Index: []


In [13]:
FifaData = pd.read_csv('C:/Users/v-ozgenc/Desktop/ratings15.csv')
FifaData.index = FifaData['Team']
cols = list(df)
cols[1], cols[0] = cols[0], cols[1]
df = df.ix[:,cols]
df = df.reset_index(drop=True)
df["homeATT"] = np.nan
df["homeMID"] = np.nan
df["homeDEF"] = np.nan
df["homeOVR"] = np.nan
df["awayATT"] = np.nan
df["awayMID"] = np.nan
df["awayDEF"] = np.nan
df["awayOVR"] = np.nan
fifateams = list(FifaData.Team)
teams = list(FullData.Team)
teams1 = list(df.home)
teams2 = list(df.away)
print(df.dtypes)
df

Season        float64
Difference    float64
away           object
home           object
score          object
week          float64
winner         object
homeATT       float64
homeMID       float64
homeDEF       float64
homeOVR       float64
awayATT       float64
awayMID       float64
awayDEF       float64
awayOVR       float64
dtype: object


c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,homeOVR,awayATT,awayMID,awayDEF,awayOVR
0,2015.0,2.0,ESKIŞEHIRSPOR,FENERBAHÇE A.Ş.,2 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015.0,-1.0,ANTALYASPOR A.Ş.,MEDIPOL BAŞAKŞEHIR FK,2 - 3,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015.0,1.0,BURSASPOR,TRABZONSPOR A.Ş.,1 - 0,1.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015.0,0.0,GALATASARAY A.Ş.,MEDICANA SIVASSPOR,2 - 2,1.0,draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015.0,-3.0,BEŞIKTAŞ A.Ş.,MERSIN IDMANYURDU,2 - 5,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015.0,0.0,KAYSERISPOR,OSMANLISPOR FUTBOL KULÜBÜ,1 - 1,1.0,draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015.0,-3.0,KASIMPAŞA A.Ş.,GAZIANTEPSPOR,0 - 3,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015.0,0.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,TORKU KONYASPOR,1 - 1,1.0,draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015.0,-1.0,ÇAYKUR RIZESPOR A.Ş.,GENÇLERBIRLIĞI,2 - 3,1.0,away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015.0,1.0,MEDIPOL BAŞAKŞEHIR FK,KASIMPAŞA A.Ş.,1 - 0,2.0,home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
for x in teams:
    for y in teams1:
        if (y.find(x) != -1):
            df.ix[df.home == y, 'homeworth'] = FullData.at[x, 'TeamWorth']
            df.ix[df.home == y, 'homeage'] = FullData.at[x, 'TeamAge']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
for x in teams:
    for y in teams2:
        if (y.find(x) != -1):
            df.ix[df.away == y, 'awayworth'] = FullData.at[x, 'TeamWorth']
            df.ix[df.away == y, 'awayage'] = FullData.at[x, 'TeamAge']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
            
for x in teams:
    for y in teams1:
        if (y.find(x) != -1):
            df.ix[df.home == y, 'homesupport'] = FullData.at[x, 'Support']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
for x in teams:
    for y in teams2:
        if (y.find(x) != -1):
            df.ix[df.away == y, 'awaysupport'] = FullData.at[x, 'Support']
df

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-d

,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,...,awayATT,awayMID,awayDEF,awayOVR,homeworth,homeage,awayworth,awayage,homesupport,awaysupport
0,2015.0,2.0,ESKIŞEHIRSPOR,FENERBAHÇE A.Ş.,2 - 0,1.0,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,195.38,24.5,48.63,23.9,28589.0,6815.0
1,2015.0,-1.0,ANTALYASPOR A.Ş.,MEDIPOL BAŞAKŞEHIR FK,2 - 3,1.0,away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48.00,26.3,48.95,25.1,2716.0,9362.0
2,2015.0,1.0,BURSASPOR,TRABZONSPOR A.Ş.,1 - 0,1.0,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,128.60,23.5,89.38,23.9,7696.0,11245.0
3,2015.0,0.0,GALATASARAY A.Ş.,MEDICANA SIVASSPOR,2 - 2,1.0,draw,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,42.05,27.0,136.00,25.6,4881.0,18996.0
4,2015.0,-3.0,BEŞIKTAŞ A.Ş.,MERSIN IDMANYURDU,2 - 5,1.0,away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,22.70,25.0,135.30,24.5,5116.0,18668.0
5,2015.0,0.0,KAYSERISPOR,OSMANLISPOR FUTBOL KULÜBÜ,1 - 1,1.0,draw,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,43.43,26.6,35.55,24.4,4186.0,4251.0
6,2015.0,-3.0,KASIMPAŞA A.Ş.,GAZIANTEPSPOR,0 - 3,1.0,away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,36.68,23.5,48.70,25.1,3065.0,1950.0
7,2015.0,0.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,TORKU KONYASPOR,1 - 1,1.0,draw,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,33.70,25.8,35.95,26.9,15871.0,2394.0
8,2015.0,-1.0,ÇAYKUR RIZESPOR A.Ş.,GENÇLERBIRLIĞI,2 - 3,1.0,away,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,42.85,25.4,32.63,24.3,2565.0,3314.0
9,2015.0,1.0,MEDIPOL BAŞAKŞEHIR FK,KASIMPAŞA A.Ş.,1 - 0,2.0,home,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48.70,25.1,48.00,26.3,1950.0,2716.0


In [15]:
spr = 'SPOR'
flag = False
for x in fifateams:
    for y in teams1:
        if(x.find(spr) != -1):
            flag = True
            #print(x)
            #print(flag)
            #print(x.find(spr), flag)
            x = x.replace(spr, '')
        if (y.find(x) != -1 and flag):
            #print(flag)
            x = x + spr
            flag = False
            #print(x, flag)
            df.ix[df.home == y, 'homeATT'] = FifaData.at[x, 'ATT']
            df.ix[df.home == y, 'homeMID'] = FifaData.at[x, 'MID']
            df.ix[df.home == y, 'homeDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.home == y, 'homeOVR'] = FifaData.at[x, 'OVR']
        elif (y.find(x) != -1):
            #print('elif: ', flag)
            #print('elif: ', x, flag)
            df.ix[df.home == y, 'homeATT'] = FifaData.at[x, 'ATT']
            df.ix[df.home == y, 'homeMID'] = FifaData.at[x, 'MID']
            df.ix[df.home == y, 'homeDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.home == y, 'homeOVR'] = FifaData.at[x, 'OVR']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
        if (flag):
            x = x + spr
        flag = False
for x in fifateams:
    for y in teams2:
        if(x.find(spr) != -1):
            flag = True
            #print(x)
            #print(flag)
            #print(x.find(spr), flag)
            x = x.replace(spr, '')
        if (y.find(x) != -1 and flag):
            #print(flag)
            x = x + spr
            flag = False
            #print(x, flag)
            df.ix[df.away == y, 'awayATT'] = FifaData.at[x, 'ATT']
            df.ix[df.away == y, 'awayMID'] = FifaData.at[x, 'MID']
            df.ix[df.away == y, 'awayDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.away == y, 'awayOVR'] = FifaData.at[x, 'OVR']
        elif (y.find(x) != -1):
            #print('elif: ', flag)
            #print('elif: ', x, flag)
            df.ix[df.away == y, 'awayATT'] = FifaData.at[x, 'ATT']
            df.ix[df.away == y, 'awayMID'] = FifaData.at[x, 'MID']
            df.ix[df.away == y, 'awayDEF'] = FifaData.at[x, 'DEF']
            df.ix[df.away == y, 'awayOVR'] = FifaData.at[x, 'OVR']
            #print(df[df.home == y].homeworth)
            #print(FullData.at[x, 'TeamWorth'])
        if (flag):
            x = x + spr
        flag = False

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\v-ozgenc\appdata\local\am

In [8]:
df

,Season,Difference,away,home,score,week,winner,homeATT,homeMID,homeDEF,homeOVR,awayATT,awayMID,awayDEF,awayOVR,homeworth,homeage,awayworth,awayage
0,2015.0,2.0,ESKIŞEHIRSPOR,FENERBAHÇE A.Ş.,2 - 0,1.0,home,79.0,76.0,76.0,77.0,68.0,68.0,68.0,68.0,195.38,24.5,48.63,23.9
1,2015.0,-1.0,ANTALYASPOR A.Ş.,MEDIPOL BAŞAKŞEHIR FK,2 - 3,1.0,away,69.0,72.0,71.0,71.0,73.0,69.0,69.0,71.0,48.00,26.3,48.95,25.1
2,2015.0,1.0,BURSASPOR,TRABZONSPOR A.Ş.,1 - 0,1.0,home,76.0,70.0,72.0,73.0,71.0,72.0,72.0,72.0,128.60,23.5,89.38,23.9
3,2015.0,0.0,GALATASARAY A.Ş.,MEDICANA SIVASSPOR,2 - 2,1.0,draw,67.0,67.0,69.0,68.0,70.0,75.0,73.0,75.0,42.05,27.0,136.00,25.6
4,2015.0,-3.0,BEŞIKTAŞ A.Ş.,MERSIN IDMANYURDU,2 - 5,1.0,away,72.0,67.0,66.0,67.0,79.0,77.0,75.0,77.0,22.70,25.0,135.30,24.5
5,2015.0,0.0,KAYSERISPOR,OSMANLISPOR FUTBOL KULÜBÜ,1 - 1,1.0,draw,71.0,72.0,69.0,71.0,71.0,69.0,68.0,69.0,43.43,26.6,35.55,24.4
6,2015.0,-3.0,KASIMPAŞA A.Ş.,GAZIANTEPSPOR,0 - 3,1.0,away,69.0,68.0,69.0,69.0,72.0,69.0,71.0,70.0,36.68,23.5,48.70,25.1
7,2015.0,0.0,AKHISAR BELEDIYE GENÇLIK VE SPOR,TORKU KONYASPOR,1 - 1,1.0,draw,69.0,72.0,69.0,70.0,71.0,70.0,69.0,70.0,33.70,25.8,35.95,26.9
8,2015.0,-1.0,ÇAYKUR RIZESPOR A.Ş.,GENÇLERBIRLIĞI,2 - 3,1.0,away,74.0,70.0,70.0,70.0,71.0,68.0,69.0,69.0,42.85,25.4,32.63,24.3
9,2015.0,1.0,MEDIPOL BAŞAKŞEHIR FK,KASIMPAŞA A.Ş.,1 - 0,2.0,home,72.0,69.0,71.0,70.0,69.0,72.0,71.0,71.0,48.70,25.1,48.00,26.3


In [16]:
(df[df.winner == 'draw']['winner']) = 'Draw'
dRAW = lambda x: x.replace('draw', 'Draw')
df.ix[df.winner == 'draw', 'winner'] = (df[df.winner == 'draw']['winner']).apply(dRAW)

c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\v-ozgenc\appdata\local\amlworkbench\python\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
df.to_csv('C:/Users/v-ozgenc/Desktop/Out.csv', encoding='utf-8')